In [63]:
tests = [["The cat is on the mat.", "A cat sits on the mat."], ["The dog is on the table!", "asodfasjdg jsajasj sadj sdjdfg?"]]
import numpy as np

Check similarity using bert model

In [64]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [75]:
def check_similarity(sentences):
    tokens1 = tokenizer(sentences[0], return_tensors="pt")
    tokens2 = tokenizer(sentences[1], return_tensors="pt")

    with torch.no_grad():
        embeddings1 = model(**tokens1).last_hidden_state.mean(dim=1)
        embeddings2 = model(**tokens2).last_hidden_state.mean(dim=1)

    norm_embeddings1 = embeddings1 / np.linalg.norm(embeddings1, axis=1)
    norm_embeddings2 = embeddings2 / np.linalg.norm(embeddings2, axis=1)
    similarity = cosine_similarity(norm_embeddings1, norm_embeddings2)
    return similarity
for i in tests:
    print(check_similarity(i))

[[0.87389827]]
[[0.4207521]]


Not impressing... :(

Check sentence-transformers model(SBERT)

In [59]:
# !pip install -U sentence-transformers

In [60]:
from sentence_transformers import SentenceTransformer

In [80]:
def check_similarity(sentences):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    embedding_1 = model.encode([sentences[0]], convert_to_tensor=True)
    embedding_2 = model.encode([sentences[1]], convert_to_tensor=True)
    return cosine_similarity(embedding_1, embedding_2)[0][0]

for i in tests:
    print(check_similarity(i))

0.9056173
0.060973637


Much better than bert! So, I think I choose this